In [185]:
# credit : https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#define-the-network

## Neural Networks
Neural networks can be constructed using the torch.nn package
An nn.Module contains layers, and a method forward(input) that returns the output

https://pytorch.org/docs/stable/nn.html#linear

## Define the network

Inputs : 8x1x32x32 (nSamples x nChannels x Height x Width)

Output : 8x10 (nSamples x nClasses)

In [186]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self,H,y):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(H, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # View returns a new tensor with the same data but of a different size
        x = x.view(-1,32*32)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net(32,y)
print(net)

Net(
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU(inplace)
)


You just have to define the forward function, and the backward function (where gradients are computed) is automatically defined for you using autograd. 
### You can use any of the Tensor operations in the forward function. 

In [187]:
class AnyOperatios(nn.Module):

    def __init__(self):
        super(AnyOperatios, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(32*32, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x, y, [x]):
        # View returns a new tensor with the same data but of a different size
        x = x.view(-1,32*32)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        #noise = torch.Tensor(3).normal_(0,1)
        for i in range(10):
            if torch.rand(1).item()>0.5:
                x = x*x
            else:
                x = x+x
        return x + y

net2 = AnyOperatios()
print(net2)

AnyOperatios(
  (fc1): Linear(in_features=1024, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU(inplace)
)


The learnable parameters of a model are returned by net.parameters()

In [194]:
net.parameters()

<generator object parameters at 0x7fd151d6a230>

In [189]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0703,  0.0148, -0.0134, -0.0727,  0.0938,  0.1302, -0.0579,
          0.1098, -0.1761,  0.1537]])


## Loss Function
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.

For example:

In [190]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4320)


## Backprop
To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call loss.backward(), and have a look at fc1’s bias gradients before and after the backward.

In [191]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('fc1.bias.grad before backward')
print(net.fc1.bias.grad)

loss.backward()

print('fc1.bias.grad after backward')
print(net.fc1.bias.grad)

fc1.bias.grad before backward
None
fc1.bias.grad after backward
tensor(1.00000e-02 *
       [ 0.0000, -2.5788,  0.0000, -1.4811, -0.8217,  0.0000, -1.1966,
         0.3178,  0.0000, -3.0738,  0.0000,  3.3465,  0.0000,  0.0000,
         0.0000, -1.0964,  0.0000,  0.5056,  1.0128,  0.0000,  0.0000,
        -0.1896, -1.6696,  0.0000,  0.0000, -1.1987,  0.2927,  0.0000,
         0.8849,  0.0000,  0.0000,  2.2814,  1.6716,  0.0000,  0.0000,
         1.3862,  0.0000,  0.0000,  0.0000,  0.0000, -1.1707, -0.6066,
         0.1717, -0.0806,  0.0000,  0.0000,  0.0000,  0.6546, -1.0097,
        -0.4544,  0.0000, -0.5843,  0.0000,  0.5264,  2.3708,  2.2536,
         2.2594,  0.0000,  1.4260,  1.8788, -3.4025,  0.0000,  0.6176,
         0.2025,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         2.0084, -3.2181,  0.0000, -0.4886,  0.0000,  0.0000, -2.0577,
         1.2799, -3.5213,  0.0000,  0.0000,  0.0000,  0.5658, -1.8295,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.7927,  

## Update the weights
The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

weight = weight - learning_rate * gradient
We can implement this using simple python code:

In [192]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [193]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
   # zero the gradient buffers

output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

Observe how gradient buffers had to be manually set to zero using optimizer.zero_grad(). This is because gradients are accumulated as explained in Backprop section.